In [1]:
import time
import sagemaker
from sagemaker.tensorflow import TensorFlow
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
job_name='har-tf-'+time.strftime('%Y-%m-%d-%H-%M-%S')
print(job_name)

har-tf-2021-05-24-02-37-04


In [4]:
hyperparameter_json={
    'epochs': 2,
    'batch_size': 64}

In [5]:
#https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
tf_estimator = TensorFlow(use_spot_instances=False,
                          enable_sagemaker_metrics=True,
                          entry_point='train4.py', 
                          #model_uri='s3://sagemaker-us-east-2-105064114710/har-tf-2021-05-21-16-14-23/output/model.tar.gz',
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.large',
                          framework_version='1.12', 
                          volume_size=8,
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameter_json
                         )

In [ ]:
#https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
'''
tf_estimator = TensorFlow(use_spot_instances=False,
                          enable_sagemaker_metrics=True,
                          entry_point='train3.py', 
                          source_dir='s3://sagemaker-us-east-2-105064114710/har-tf-2021-05-19-13-36-45/source/sourcedir.tar.gz',
                          model_uri='s3://sagemaker-us-east-2-105064114710/har-tf-2021-05-19-15-18-36/output/model.tar.gz',
                          #output_dir='specify'
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.large',
                          framework_version='1.12', 
                          volume_size=8,
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameter_json
                         )
'''

In [6]:
#https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

tf_estimator.fit(
    inputs={
    'training': 's3://har-data-105064114710/train',
    'test': 's3://har-data-105064114710/test'
    },
    wait=True,
    job_name=job_name)

2021-05-24 02:37:18 Starting - Starting the training job...
2021-05-24 02:37:19 Starting - Launching requested ML instancesProfilerReport-1621823837: InProgress
......
2021-05-24 02:38:33 Starting - Preparing the instances for training......
2021-05-24 02:39:34 Downloading - Downloading input data...
2021-05-24 02:40:14 Training - Training image download completed. Training in progress..2021-05-24 02:40:08,889 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-05-24 02:40:08,895 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-24 02:40:09,201 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-24 02:40:09,217 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-24 02:40:09,228 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input


2021-05-24 02:41:37 Uploading - Uploading generated training model
2021-05-24 02:41:37 Completed - Training job completed
Training seconds: 131
Billable seconds: 131


In [8]:
#https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html

import boto3
import json
import pprint
client = boto3.client('sagemaker')

response = client.describe_training_job(
    TrainingJobName=job_name
)

while (response['TrainingJobStatus'] not in ('Failed','Completed','Stopped','Stopping','Interrupted','MaxRuntimeExceeded')):
    print (response['TrainingJobStatus'])
    time.sleep(15)
    response = client.describe_training_job(TrainingJobName=job_name)
    

#pprint.pprint(response)
print(job_name)
print(response['TrainingTimeInSeconds'])
print(response['TrainingJobStatus'])
print(response['ModelArtifacts']['S3ModelArtifacts'])

har-tf-2021-05-24-02-37-04
131
Completed
s3://sagemaker-us-east-2-105064114710/har-tf-2021-05-24-02-37-04/output/model.tar.gz


In [9]:
#https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html#deploying-directly-from-model-artifacts

from sagemaker.tensorflow import TensorFlowModel
tf_endpoint_name = 'ep-'+job_name

model_artifact = response['ModelArtifacts']['S3ModelArtifacts']

model = TensorFlowModel(model_data=model_artifact, role=role,framework_version='1.12')
                                       
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.t2.medium',
                         endpoint_name=tf_endpoint_name,
                         #accelerator_type='ml.eia1.medium'
                         wait=False)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [10]:
#https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html

import boto3
import json
import pprint
client = boto3.client('sagemaker')

response = client.describe_endpoint(
    EndpointName=tf_endpoint_name
)

while (response['EndpointStatus'] not in ('InService')):
    time.sleep(10)
    response = client.describe_endpoint(
    EndpointName=tf_endpoint_name)
    print(response['EndpointStatus'])
    
pprint.pprint(response)


Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'CreationTime': datetime.datetime(2021, 5, 24, 2, 43, 26, 846000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:105064114710:endpoint/ep-har-tf-2021-05-24-02-37-04',
 'EndpointConfigName': 'ep-har-tf-2021-05-24-02-37-04',
 'EndpointName': 'ep-har-tf-2021-05-24-02-37-04',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2021, 5, 24, 2, 50, 39, 324000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2021, 5, 24, 2, 43, 29, 3000, tzinfo=tzlocal()),
                                             'ResolvedImage': '520713654638.dkr.ecr.us-east-2.amazonaws.com/sagemaker-tensorflow-serving@sha256:026c2a51a2e9e258481b17c110f9c9640dc010bf71463d4bd31